# ReLU

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

### Problem of Sigmoid

양 끝의 gradient가 잘 안나와서 loss로부터 전파되는 gradient가 소멸된다 => Vanishing Gradient

### ReLU

x의 값에 따라(0을 기준으로) gradient를 0 또는 1으로

### Optimizer

  
### ReLU Code

###### Review : reading data

In [4]:
# MNIST dataset
mnist_train = dsets.MNIST(root="MNIST_data/", 
                          train=True, 
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root="MNIST_data/",
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)

In [7]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [18]:
# MNIST data image of shape 28 * 28 = 784
linear = torch.nn.Linear(784, 10, bias=True).to(device)   

In [19]:
# Initialization
torch.nn.init.normal_(linear.weight)

Parameter containing:
tensor([[ 0.4408,  0.0333,  1.2665,  ...,  1.3655,  0.9023,  0.2711],
        [-0.8514, -1.0118,  1.3863,  ...,  0.4555,  0.1585, -0.0660],
        [-0.3485,  0.7535,  0.6964,  ...,  1.2294,  0.1562, -0.6095],
        ...,
        [-1.8461, -0.3174, -0.9629,  ...,  0.9100,  0.4152,  0.5850],
        [ 1.1215,  1.0075,  0.9530,  ..., -0.5068,  0.8228,  0.3259],
        [-1.4698, -1.7320, -0.1814,  ..., -0.8279,  1.6980,  1.9932]],
       device='cuda:0', requires_grad=True)

In [20]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(linear.parameters(), lr=learning_rate)

In [21]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = linear(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 5.003168106
Epoch: 0002 cost = 1.567474842
Epoch: 0003 cost = 1.022292733
Epoch: 0004 cost = 0.811989307
Epoch: 0005 cost = 0.698020875
Epoch: 0006 cost = 0.624068677
Epoch: 0007 cost = 0.571655214
Epoch: 0008 cost = 0.531443298
Epoch: 0009 cost = 0.499901205
Epoch: 0010 cost = 0.474057913
Epoch: 0011 cost = 0.452617437
Epoch: 0012 cost = 0.434431225
Epoch: 0013 cost = 0.419008970
Epoch: 0014 cost = 0.404837996
Epoch: 0015 cost = 0.393492192
Learning finished


In [22]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = linear(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = linear(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.8915666937828064
Label:  4
Prediction:  4
